In [2]:
pip install google_images_download

## I have used google_images_download liabrary to scrap dataset form google image search.

In [3]:
from google_images_download import google_images_download   #importing the library

response = google_images_download.googleimagesdownload()   #class instantiation

arguments = {"keywords":"Wagon R,Swift,Santro,i10","limit":50,"print_urls":True}   #creating list of arguments
paths = response.download(arguments)   #passing the arguments to the function
print(paths)


Item no.: 1 --> Item name = Wagon R
Evaluating...
Starting Download...
Image URL: https://www.marutisuzuki.com//marutiprodcdn.azureedge.net/-/media/images/maruti/marutisuzuki/car/car-profile-shots/newwagonr/poolside-blue.ashx?modified=20190906080602
Invalid or missing image format. Skipping...
Image URL: https://auto.ndtvimg.com/car-images/large/maruti-suzuki/wagon-r/maruti-suzuki-wagon-r.jpg?v=40
Completed Image ====> 1.maruti-suzuki-wagon-r.jpg
Image URL: https://marutiprodcdn.azureedge.net/-/media/images/maruti/marutisuzuki/car/car-specs-image/new_wagonr/wagon_spec.ashx?modified=20190429124421
Invalid or missing image format. Skipping...
Image URL: https://cdni.autocarindia.com/Utils/ImageResizer.ashx?n=http%3A%2F%2Fcdni.autocarindia.com%2FExtraImages%2F20191120121413_Maruti-Wagon-R.jpg&h=578&w=872&c=0
Invalid or missing image format. Skipping...
Image URL: https://stimg.cardekho.com/images/carexteriorimages/360x240/Maruti/Maruti-Wagon-R/1548228678044/047.jpg
Completed Image ====> 

In [4]:
from keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = 300
WIDTH = 300

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [5]:
from keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = "downloads"
HEIGHT = 300
WIDTH = 300
BATCH_SIZE = 8

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)

Found 343 images belonging to 4 classes.


In [6]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = ["Wagon R","Swift","Santro","i10"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
!mkdir ./checkpoints
!touch ./checkpoints/ResNet50_model_weights.h5

mkdir: cannot create directory ‘./checkpoints’: File exists


In [8]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

NUM_EPOCHS = 10
BATCH_SIZE = 8
num_train_images = 10000

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath="./checkpoints/" + "ResNet50" + "_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
callbacks_list = [checkpoint]

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       shuffle=True, callbacks=callbacks_list)





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


1250/1250 [==============================] - 407s 326ms/step - loss: 1.2163 - acc: 0.5833

Epoch 00001: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 2/10
1250/1250 [==============================] - 399s 320ms/step - loss: 0.4423 - acc: 0.8359

Epoch 00002: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 3/10
1250/1250 [==============================] - 398s 318ms/step - loss: 0.2431 - acc: 0.9147

Epoch 00003: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 4/10
1250/1250 [==============================] - 396s 317ms/step - loss: 0.1588 - acc: 0.9436

Epoch 00004: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 5/10
1250/1250 [==============================] - 396s 317ms/step - loss: 0.1229 - acc: 0.9578

Epoch 00005: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 6/10
1250/1250 [==============================] - 396s 317ms/step - loss: 0.0963 - acc: 0.9681

Epoch 00006: saving model to ./checkpoints/ResNet50_mo

In [0]:
finetune_model.save("model.h5")